In [1]:
import json
import pandas

In [3]:
raw_ra_sales_transactions_dataframe = pandas.read_excel('data/royal/royal_apothecary_sales_transactions_20210101_20210905.xlsx', header=0)
raw_ra_sales_transactions_dataframe.columns

Index(['date_type', 'id', 'last_modified_at', 'receipt_id', 'receipt_number',
       'receipt_type', 'sales_customer_type', 'sales_datetime',
       'total_packages', 'total_price', 'tx_type', 'tx_package_id',
       'tx_package_label', 'tx_product_name', 'tx_product_category_name',
       'tx_unit_of_measure', 'tx_quantity_sold', 'tx_total_price'],
      dtype='object')

In [4]:
raw_hpcc_sales_transactions_dataframe = pandas.read_excel('data/hpcc_sales_transactions_20210601_20210918.xlsx', header=0)
raw_hpcc_sales_transactions_dataframe.columns

Index(['date_type', 'id', 'last_modified_at', 'receipt_id', 'receipt_number',
       'receipt_type', 'sales_customer_type', 'sales_datetime',
       'total_packages', 'total_price', 'tx_type', 'tx_package_id',
       'tx_package_label', 'tx_product_name', 'tx_product_category_name',
       'tx_unit_of_measure', 'tx_quantity_sold', 'tx_total_price'],
      dtype='object')

In [5]:
ra_sales_transactions_records = raw_ra_sales_transactions_dataframe.to_dict('records')
hpcc_sales_transactions_records = raw_hpcc_sales_transactions_dataframe.to_dict('records')

In [18]:
ra_product_name_to_sales_transaction = dict({})
for ra_sales_transaction_record in ra_sales_transactions_records:
    tx_product_name = ra_sales_transaction_record['tx_product_name']
    mini_sales_transaction = {
        'sales_datetime': ra_sales_transaction_record['sales_datetime'],
        'tx_quantity_sold': ra_sales_transaction_record['tx_quantity_sold'],
    }
    if tx_product_name in ra_product_name_to_sales_transaction:
        ra_product_name_to_sales_transaction[tx_product_name].append(mini_sales_transaction)
    else:
        ra_product_name_to_sales_transaction[tx_product_name] = [mini_sales_transaction]

In [19]:
len(list(ra_product_name_to_sales_transaction.keys()))

1560

In [26]:
total_count = 0
match_count = 0

match_product_names_set = set([])

for hpcc_sales_transaction_record in hpcc_sales_transactions_records:
    tx_product_name = hpcc_sales_transaction_record['tx_product_name']

    if tx_product_name in ra_product_name_to_sales_transaction:
        match_count += 1
        match_product_names_set.add(tx_product_name)

    total_count += 1

match_count, total_count, list(match_product_names_set)[:10]

(45125,
 92412,
 [nan,
  'Korova Mini Dip - Chocolate Chip Dip - 100mg / Case of 30',
  'Rich & Ruthless - CPT Glue (14.0 GM Flower (Each)) (#534-CPTG-OC0821) #34331',
  'Stone Road Concentrate Do-Si-Dos Sauce',
  'STIIIZY - Strawnana Pod - 1.0g',
  'Kushy Punch Sativa Gummy 100mg',
  'Cali Flwr Farms - Sour Grape Gummies (66 GM Edible (weight - each)) (#06-CFSG-0421) #33590',
  'Kushy Punch Sugar Free Sativa Gummy 100mg',
  'STIIIZY - Premium Jack Pod - 0.5g',
  'Viva Sativa - Viva Fuego (28 GM Flower (Each)) (#529-VSVF-OC0821) #34303'])

In [13]:
f'{match_count} of {total_count} of SKUs sold by HPCC from 06/21 to 09/18 were also sold by RA in 2021 ({match_count / total_count * 100}%)'

'45125 of 92412 of SKUs sold by HPCC from 06/21 to 09/18 were also sold by RA in 2021 (48.83023849716487%)'

In [30]:
for match_product_name in list(match_product_names_set)[:25]:
    print(match_product_name, ra_product_name_to_sales_transaction[match_product_name])
    print('-----------')

nan [{'sales_datetime': Timestamp('2021-08-20 18:51:38.010000'), 'tx_quantity_sold': nan}]
-----------
Korova Mini Dip - Chocolate Chip Dip - 100mg / Case of 30 [{'sales_datetime': Timestamp('2021-07-16 10:07:52.790000'), 'tx_quantity_sold': 3.0}, {'sales_datetime': Timestamp('2021-07-08 17:31:10.330000'), 'tx_quantity_sold': 1.0}, {'sales_datetime': Timestamp('2021-07-08 16:05:10.460000'), 'tx_quantity_sold': 2.0}, {'sales_datetime': Timestamp('2021-06-07 10:09:43.640000'), 'tx_quantity_sold': 2.0}]
-----------
Rich & Ruthless - CPT Glue (14.0 GM Flower (Each)) (#534-CPTG-OC0821) #34331 [{'sales_datetime': Timestamp('2021-08-28 16:53:51.320000'), 'tx_quantity_sold': 1.0}]
-----------
Stone Road Concentrate Do-Si-Dos Sauce [{'sales_datetime': Timestamp('2021-08-29 13:10:49.860000'), 'tx_quantity_sold': 1.0}, {'sales_datetime': Timestamp('2021-08-27 13:32:19.890000'), 'tx_quantity_sold': 1.0}, {'sales_datetime': Timestamp('2021-08-26 15:37:26.480000'), 'tx_quantity_sold': 1.0}, {'sales_